In [1]:
import pandas as pd
df=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
df.head()

,Id,headlines,category
0,0,উত্তর প্রদেশে প্রিয়াংকার ইমেজ ব্যর্থ,0
1,1,১০০০ বিড়াল অতিথিকে নিয়ে বিয়ে করলেন কানাডিয়া...,1
2,2,মিসর ছেড়ে চেচনিয়ার হয়ে খেলবেন সালাহ!,0
3,3,আলেপ্পোয় বিমান হামলা যুদ্ধাপরাধের শামিল,0
4,4,"সিয়াটলে চুরি হওয়া বিমানটি বিধ্বস্ত, চালক ছিলেন...",0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48508 entries, 0 to 48507
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Id         48508 non-null  int64 
 1   headlines  48505 non-null  object
 2   category   48508 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [3]:
df.shape

(48508, 3)

In [4]:
test

,Id,headlines
0,0,"মেসি-রোনাল্ডোর দিন শেষ, ব্যালন ডিঅর পাচ্ছেন মড..."
1,1,রানি মুখার্জির ফেরা নিশ্চিত হল
2,2,মোদি মিথ্যাবাদী : গগৈ
3,3,"সন্ত্রাস, দুর্নীতিতে জড়িত কাউকে ছাড় দেয়া হবে ন..."
4,4,আবারো শেষের নায়ক রামোস
...,...,...
12122,12122,প্রথম ক্ষেপণাস্ত্র সজ্জিত সাবমেরিন উন্মোচন ইরানের
12123,12123,তলিয়ে গেছে নগরী
12124,12124,নাসায় কর্মরত ১৭-র বিস্ময় বালক
12125,12125,বিএনপি-জামায়াতের ষড়যন্ত্রেই কারণে গণহত্যার আন্...


# checking duplicate values


In [5]:
df['headlines'].duplicated().sum()

1372

In [6]:
df=df.drop_duplicates(subset='headlines')

In [7]:
labels = (df['category'])
sentences = (df['headlines'].astype(str))
test1=(df['Id'])

print(sentences[0])

উত্তর প্রদেশে প্রিয়াংকার ইমেজ ব্যর্থ


In [8]:
training_size=35000
import csv
import tensorflow as tf


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 50000
embedding_dim = 300

trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

tokenizer = Tokenizer(num_words=50000, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
max_length =max([len(i) for i in training_sequences]) 

training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing1_sequences=tokenizer.texts_to_sequences(test['headlines'])

testing1_padded=pad_sequences(testing1_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)

testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [9]:
testing_padded

array([[23955,     1,  2394, ...,     0,     0,     0],
       [  240,  1205,   512, ...,     0,     0,     0],
       [  311,     1,  1971, ...,     0,     0,     0],
       ...,
       [   61,   154, 10203, ...,     0,     0,     0],
       [ 2509,  2559,     1, ...,     0,     0,     0],
       [ 9655,  3021,  2581, ...,     0,     0,     0]])

In [10]:
vocab_size = len(tokenizer.word_index)

In [11]:
import numpy as np
training_padded = np.array(training_padded)

training_labels = np.array(training_labels)

testing_padded = np.array(testing_padded)

testing_labels = np.array(testing_labels)

testing1_padded=np.array(testing1_padded)

print(training_padded.shape)
print(training_labels.shape)

(35000, 21)
(35000,)


In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
     tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.50),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [16]:
num_epochs = 10
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), 
                    verbose=2)

Epoch 1/10
1094/1094 - 158s - loss: 0.2681 - accuracy: 0.8986 - val_loss: 0.1945 - val_accuracy: 0.9238 - 158s/epoch - 145ms/step
Epoch 2/10
1094/1094 - 660s - loss: 0.1245 - accuracy: 0.9558 - val_loss: 0.1784 - val_accuracy: 0.9335 - 660s/epoch - 603ms/step
Epoch 3/10
1094/1094 - 162s - loss: 0.0693 - accuracy: 0.9774 - val_loss: 0.1972 - val_accuracy: 0.9298 - 162s/epoch - 148ms/step
Epoch 4/10
1094/1094 - 160s - loss: 0.0390 - accuracy: 0.9881 - val_loss: 0.2388 - val_accuracy: 0.9308 - 160s/epoch - 146ms/step
Epoch 5/10
1094/1094 - 315s - loss: 0.0247 - accuracy: 0.9926 - val_loss: 0.2637 - val_accuracy: 0.9234 - 315s/epoch - 288ms/step
Epoch 6/10
1094/1094 - 351s - loss: 0.0147 - accuracy: 0.9957 - val_loss: 0.3100 - val_accuracy: 0.9271 - 351s/epoch - 320ms/step
Epoch 7/10
1094/1094 - 164s - loss: 0.0098 - accuracy: 0.9970 - val_loss: 0.3748 - val_accuracy: 0.9183 - 164s/epoch - 150ms/step
Epoch 8/10
1094/1094 - 158s - loss: 0.0067 - accuracy: 0.9979 - val_loss: 0.3767 - val_acc

In [17]:
prediction=model.predict(testing1_padded)
print('Outputs shape')    
print(prediction.shape) # prints (n,1) but  need (n,)
prediction=np.transpose(prediction)[0]  # transformation to get (n,)
print(prediction.shape)  # now the shape is (n,)
# Applying transformation to get binary values predictions with 0.5 as thresold
prediction = list(map(lambda x: 0 if x<0.5 else 1,prediction))

379/379 [==============================] - 0s 895us/step
Outputs shape
(12127, 1)
(12127,)


In [18]:
submission=test[['Id']]
submission

,Id
0,0
1,1
2,2
3,3
4,4
...,...
12122,12122
12123,12123
12124,12124
12125,12125


In [19]:
submission['category']=prediction

In [20]:
submission

,Id,category
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
12122,12122,0
12123,12123,0
12124,12124,1
12125,12125,0


In [23]:
submission.to_csv("Datalyzed_16-23.csv",index=None)